In [13]:
import pandas as pd
pd.set_option('mode.chained_assignment',  None)
kogpt_dataset_1_1 = pd.read_csv("/opt/ml/dataset/train/kogpt_train_ing.csv", index_col = 0, encoding='utf-8')
kogpt_dataset_1_1

,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,"〈Something〉는 조지 해리슨이 쓰고 비틀즈가 녹음한 노래다. 노래는 피터, ...",비틀즈,조지 해리슨,ORG,PER,no_relation,"[24, 26]","[13, 18]"
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 4·15 총선을 앞두고 통합을 추진하...,민주평화당,대안신당,ORG,ORG,no_relation,"[19, 23]","[14, 17]"
2,2,"K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹에 ""구...",광주FC,한국프로축구연맹,ORG,ORG,org:member_of,"[21, 24]","[34, 41]"
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)가 23일 지구의 날을 맞아 다양한 ...,아성다이소,박정부,ORG,PER,org:top_members/employees,"[13, 17]","[22, 24]"
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에 입단하여 프로 생활을 시...,요미우리 자이언츠,1967,ORG,DAT,no_relation,"[22, 30]","[0, 3]"
...,...,...,...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,유기준,부산 서구·동구,PER,LOC,per:employee_of,"[93, 95]","[100, 107]"
32466,32466,"법포는 다시 최시형, 서병학, 손병희 등으로 이어져 1905년 1월 사상 최초의 정...",최시형,손병희,PER,PER,per:colleagues,"[7, 9]","[17, 19]"
32467,32467,완도군(군수 신우철)은 지난 6일 완도군청 상황실에서 「완도자연그대로 농・축산업」 ...,완도군,신우철,ORG,PER,org:top_members/employees,"[0, 2]","[7, 9]"
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장으로 재직하며 '중앙일보-JTB...",JTBC,중앙홀딩스,ORG,ORG,no_relation,"[6, 9]","[21, 25]"


In [14]:
vanila_dataset = pd.read_csv("/opt/ml/dataset/train/train.csv", index_col = 0, encoding = 'utf-8')
vanila_dataset.reset_index(drop=False, inplace=True)

data= vanila_dataset
        
sub_entity, sub_type= [], []
obj_entity, obj_type= [], []
sub_idx, obj_idx= [], []
sentence= []

"""preprocess"""
for i, [x, y, z] in enumerate(zip(data['subject_entity'], data['object_entity'], data['sentence'])):
    sub_typ= x[1:-1].split(':')[-1].split('\'')[-2]
    obj_typ= y[1:-1].split(':')[-1].split('\'')[-2]
            
    for idx_i in range(len(x)):
        if x[idx_i: idx_i+ 9]== 'start_idx':
            sub_start= int(x[idx_i+12:].split(',')[0].strip())
        if x[idx_i: idx_i+7]== 'end_idx':
            sub_end= int(x[idx_i+10:].split(',')[0].strip())
                
        if y[idx_i: idx_i+ 9]== 'start_idx':
            obj_start= int(y[idx_i+12:].split(',')[0].strip())
        if y[idx_i: idx_i+7]== 'end_idx':
            obj_end= int(y[idx_i+10:].split(',')[0].strip())
            
    sub_i= [sub_start, sub_end]
    obj_i= [obj_start, obj_end]

    sub_entity.append(z[sub_i[0]: sub_i[1]+1])
    obj_entity.append(z[obj_i[0]: obj_i[1]+1])
    sub_type.append(sub_typ); sub_idx.append(sub_i)
    obj_type.append(obj_typ); obj_idx.append(obj_i)
    
    z = z[min(sub_i[0],obj_i[0]) : max(sub_i[1],obj_i[1])+1]
    sentence.append(z)

vanila_dataset = pd.DataFrame({'id': data['id'], 'sentence' : sentence, 'subject_entity': sub_entity, 'object_entity': obj_entity,
                        'subject_type': sub_type, 'object_type': obj_type, 'label': data['label'],
                        'subject_idx': sub_idx, 'object_idx': obj_idx})

vanila_dataset

,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,조지 해리슨이 쓰고 비틀즈,비틀즈,조지 해리슨,ORG,PER,no_relation,"[24, 26]","[13, 18]"
1,1,대안신당·민주평화당,민주평화당,대안신당,ORG,ORG,no_relation,"[19, 23]","[14, 17]"
2,2,광주FC는 지난 26일 한국프로축구연맹,광주FC,한국프로축구연맹,ORG,ORG,org:member_of,"[21, 24]","[34, 41]"
3,3,아성다이소(대표 박정부,아성다이소,박정부,ORG,PER,org:top_members/employees,"[13, 17]","[22, 24]"
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠,요미우리 자이언츠,1967,ORG,DAT,no_relation,"[22, 30]","[0, 3]"
...,...,...,...,...,...,...,...,...,...
32465,32465,유기준(4선·부산 서구·동구,유기준,부산 서구·동구,PER,LOC,per:employee_of,"[93, 95]","[100, 107]"
32466,32466,"최시형, 서병학, 손병희",최시형,손병희,PER,PER,per:colleagues,"[7, 9]","[17, 19]"
32467,32467,완도군(군수 신우철,완도군,신우철,ORG,PER,org:top_members/employees,"[0, 2]","[7, 9]"
32468,32468,JTBC 회장을 지낸 이후 중앙홀딩스,JTBC,중앙홀딩스,ORG,ORG,no_relation,"[6, 9]","[21, 25]"


In [15]:
mod_list = []
for i in range(len(vanila_dataset)) :
    if vanila_dataset['sentence'][i] not in kogpt_dataset_1_1['sentence'][i] :
        # kogpt_dataset_1_1['sentence'][i] = vanila_dataset['sentence'][i]
        mod_list.append(i)
print(mod_list)

[1963, 2379, 3274, 3508, 4812, 4815, 4839, 4883, 4890, 4894, 4895, 4899, 4952, 4960, 4974, 4977, 4984, 5001, 5003, 5020, 5041, 5066, 5076, 5092, 5123, 5133, 5163, 5175, 5182, 5183, 5222, 5264, 5317, 5326, 5327, 5329, 5344, 5354, 5356, 5369, 5380, 5383, 5390, 5401, 5421, 5423, 5494, 5511, 5533, 5541, 5548, 5556, 5577, 5604, 5650, 5663, 5771, 5804, 5863, 5864, 5873, 5884, 5972, 5974, 5989, 5993, 5995, 6012, 6046, 6089, 6125, 6137, 6146, 6178, 6183, 6191, 6207, 6246, 6247, 6303, 6312, 6320, 6332, 6334, 6350, 6352, 6356, 6358, 6393, 6395, 6399, 6423, 6453, 6533, 6534, 6539, 6540, 6567, 6623, 6656, 6663, 6671, 6682, 6701, 6709, 6728, 6752, 6763, 6769, 6774, 6776, 6783, 6802, 6819, 6825, 6839, 6851, 6858, 6864, 6871, 6879, 6880, 6895, 6923, 6939, 6941, 6973, 7080, 7116, 7138, 7145, 7155, 7194, 7212, 7226, 7280, 7296, 7304, 7307, 7318, 7323, 7327, 7331, 7355, 7367, 7398, 7408, 7411, 7414, 7441, 7449, 7452, 7483, 7519, 7520, 7523, 7526, 7530, 7558, 7564, 7568, 7573, 7582, 7591, 7600, 7603, 761

In [16]:
print(kogpt_dataset_1_1['subject_entity'][31898])
print(kogpt_dataset_1_1['object_entity'][31898])
print(kogpt_dataset_1_1['sentence'][31898])
print(vanila_dataset['sentence'][31898])

레알 마드리드
라리가
2012년 3월 31일, 그는 오사수나와의 경기에서 자신의 100번째 레알 마드리드 골을 기록했고, 한 골 더 넣어 라리가 역대 최다 골 기록을 경신하게 되었다. 이 골은 100골을 기록한 후 레알 마드리드의 홈인 산티아고 베르나베
라리가 2011-12 시즌에 레알 마드리드


In [17]:
#-*- encoding: utf-8 -*-
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.notebook import tqdm

tokenizer = AutoTokenizer.from_pretrained('kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
                                          bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]')
model = AutoModelForCausalLM.from_pretrained('kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',pad_token_id=tokenizer.eos_token_id,
    torch_dtype='auto', low_cpu_mem_usage=True).to(device='cuda', non_blocking=True)

_ = model.eval()

for i in tqdm(mod_list):
    prompt = vanila_dataset['sentence'][i]
    with torch.no_grad():
        tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
        gen_tokens = model.generate(tokens ,do_sample=True, temperature=0.8, max_length = 128)
        generated = tokenizer.batch_decode(gen_tokens)[0]

    kogpt_dataset_1_1['sentence'][i] = generated
    kogpt_dataset_1_1.to_csv("/opt/ml/dataset/train/kogpt_train_ing.csv", encoding = 'utf-8')

Input length of input_ids is 137, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
